In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, AveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import os
import random

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)


parent_dataset_folder = "/content/drive/MyDrive/Dataset"

target_size = (224, 224)

def preprocess_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    #normalizing pixel values
    image_array = image_array / 255.0
    return image_array


def load_and_preprocess_images(directory):
    images = []
    labels = []
    for class_folder in os.listdir(directory):
        class_folder_path = os.path.join(directory, class_folder)
        if os.path.isdir(class_folder_path):
            for filename in os.listdir(class_folder_path):
                if filename.endswith(".jpg") or filename.endswith(".png"):
                    image_path = os.path.join(class_folder_path, filename)
                    image = preprocess_image(image_path)
                    images.append(image)
                    labels.append(class_folder)
    return np.array(images), np.array(labels)

preprocessed_images, labels = load_and_preprocess_images(parent_dataset_folder)

#encode labels
label_binarizer = LabelBinarizer()
labels_encoded = label_binarizer.fit_transform(labels)

X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(preprocessed_images, labels_encoded, test_size=0.2, random_state=42)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = AveragePooling2D(pool_size=(7, 7))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax', name='dense_1_of_2')(x)  #output layer with 2 neurons

model = Model(inputs=base_model.input, outputs=predictions)

#freezing the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(X_train, y_train_encoded, batch_size=44, epochs=74, validation_data=(X_test, y_test_encoded))

loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


9406464/9406464 [==============================] - 0s 0us/step


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [3]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


y_pred_encoded = model.predict(X_test)

y_pred_labels = np.argmax(y_pred_encoded, axis=1)

#classification report
report = classification_report(y_test_encoded, y_pred_labels, target_names=label_binarizer.classes_)
print(report)

conf_matrix = confusion_matrix(y_test_encoded, y_pred_labels)
print(conf_matrix)


21/21 [==============================] - 5s 196ms/step
              precision    recall  f1-score   support

     Healthy       0.97      0.98      0.97       304
   Parkinson       0.98      0.97      0.98       349

    accuracy                           0.98       653
   macro avg       0.98      0.98      0.98       653
weighted avg       0.98      0.98      0.98       653

[[297   7]
 [  9 340]]
